In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def grad_LJ(r, eps, sig):
    n = len(r)
    grad = np.zeros_like(r, dtype="float64")
    for i in range(0,n):
        for j in range(0, n):
            if i != j:
                dr = r[i] - r[j]
                length = np.linalg.norm(dr)
                grad[i] += eps*((6*sig**6)/length**8 - (12*sig**12)/length**12)*dr
    return grad 

In [ ]:
def grad_harmonic(r, k):
    return 2*k*r

In [ ]:
def potential_gradient(r, eps=1, sig=1, k=1):
    return (grad_LJ(r, eps, sig) + grad_harmonic(r, k))

In [ ]:
def vv(potential_gradient, r_init, v_init, dt=1/100, t_ges=100, eps=1., sig=1., k=1.):
    
    size = int(t_ges/dt)
    n = len(r_init)
    
    r_matrix, v_matrix = np.zeros((size, n, 2)), np.zeros((size, n, 2))
    r_matrix[0], v_matrix[0] = r_init, v_init
    
    for i in range(1,size):
        
        r = r_matrix[i-1]
        v = v_matrix[i-1]
        gradient = potential_gradient(r, eps, sig, k)
        
        r_new = r + dt * v - 1/2 * dt**2 * gradient
        gradient_new = potential_gradient(r_new, eps, sig, k) 
        v_new = v - 1/2 * dt * (gradient + gradient_new)
        
        r_matrix[i], v_matrix[i] = r_new, v_new

        
    return r_matrix, v_matrix

In [ ]:
r_init = np.random.uniform(low=-5, high=5,size=(4,2))
v_init = np.zeros_like(r_init)
r_matrix, v_matrix = vv(potential_gradient, r_init, v_init, k=1/20)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
scat = ax.scatter(r_matrix[0,:,0], r_matrix[0,:,1], c=np.arange(len(r_init)))
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=20, width=0.005)
ax.set_xlim(np.max([-10,np.min(r_matrix[:,:,0])]),np.min([10,np.max(r_matrix[:,:,0])]))
ax.set_ylim(np.max([-10,np.min(r_matrix[:,:,1])]),np.min([10,np.max(r_matrix[:,:,1])]))

def animate(i):
    index = 2*i
    data = r_matrix[index]
    scat.set_offsets(data)
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)

anim = animation.FuncAnimation(fig, animate, interval=40, blit=True, repeat=False)

In [ ]:
def gd(potential_gradient, r_init, gamma=1/100, size=10000, eps=1, sig=1, k=1):
    n = len(r_init)
    r_ = r_init
    for i in range(1,size):
        r = r_ -gamma*potential_gradient(r_, eps, sig, k)
        r_ = r[::]
    return r  

In [ ]:
r_init = np.random.uniform(low=-1, high=1,size=(3,2))
r = gd(potential_gradient,r_init, k=1/10)
plt.scatter(r[:,0],r[:,1])
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.show()
    